In [ ]:
from Env_c import Env
import os
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import TD3
from stable_baselines3.common.monitor import Monitor

#### Testing out the environment

In [ ]:
env = Env(True)
env.reset()

In [ ]:
for i in range(10):

    done = False
    env.reset()
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        print("Action: ", action, "Reward", reward, "Done", done, "Distance Rate", env.distance_rate, "Heading", env.heading)

        # print("State: ", n_state)


#### Training

In [ ]:
models_dir = "models/midas"
logdir = "logs"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):  
    os.makedirs(logdir)

In [ ]:
env = Env(True)
# env = Monitor(env, filename="log.txt" ,info_keywords=("collide", "goal", 'limit'))
env.reset()

In [ ]:

path = 'models/midas/midas_50000_steps'
model = TD3.load(path, env=env)
# model = TD3("MlpPolicy", env, seed=0, verbose=1, tensorboard_log=logdir)

In [ ]:
# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path=models_dir,
                                         name_prefix='midas')

In [ ]:
model.learn(total_timesteps=300_000, reset_num_timesteps=False, tb_log_name="midas", callback=checkpoint_callback)

### Eval

In [ ]:
env = Env(True)
env = Monitor(env, filename="log.txt" ,info_keywords=("collide", "goal", 'limit'))

env.reset()
# PPO_path = 'models/TD3/TD3_model_375000_steps'
# model = TD3.load(PPO_path, env=env)
model = TD3("MlpPolicy", env, verbose=1)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=1, deterministic=True)

### Test

In [ ]:
env = Env(False)
PPO_path = 'models/TD3/TD3_model_375000_steps'
model = TD3.load(PPO_path, env=env)
# model = PPO("MlpPolicy", env, verbose=1)

In [ ]:
t_reward = 0

for i in range(1):

    done = False
    obs = env.reset()
    # time.sleep(10)

    
    while not done:
        # action, _ = model.predict(obs, deterministic=True)
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        t_reward += reward
        # print("Action: ", action, "Reward", reward, "Done", done, "Distance Rate", env.distance_rate*500, "Heading", env.heading)
        # print("Distance Rate", env.distance_rate*1, "Current Distance", env.current_distance, "Heading", env.heading)
        print(action)
        # print("State: ", n_state)

print(t_reward)


In [ ]:
for i in range(10):
    obs, reward, done, info = env.step(3)
